In [1]:
from utility_funcs import get_train_labels_test, split_train_data, scale_and_as_array

import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm



In [2]:
train, labels, test = get_train_labels_test()

In [3]:
features = [f for f in train.columns if 'sensor' in f]

In [4]:
groups = train["sequence"]
train = train.drop(["subject", "step",'sequence'], axis=1)
test = test.drop([ "subject", "step",'sequence'], axis=1)

In [5]:

X_train, X_test, y_train, y_test = split_train_data(train, labels)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
test, X_train, X_test, y_train, y_test = scale_and_as_array(train, features, test, X_train, X_test,y_train, y_test)

(1402260, 13) (23371,)
(155820, 13) (2597,)


In [6]:
class CustomDataset(Dataset):
    def __init__(self,X, sequence_len, y = None, mode = 'train'):
        self.data = X
        self.target = y
        self.sequence_len = sequence_len
        self.mode = mode
    
    def __len__(self):
        return (self.data.shape[0]//60) # each sequence is of length 60 
    
    def __getitem__(self, idx):
        out_x = self.data[idx]
        if self.mode == 'train':
            out_y =  self.target[idx[0]//self.sequence_len]
            return out_x, out_y
        else: 
            return out_x


def CustomDataloader(dataset,  dataset_num, sequence_len,  input_size, batch_size):
    sampler = np.array([list(range(i*sequence_len, (i+1)*sequence_len)) for i in range(dataset_num//sequence_len)])
    dataloader = DataLoader(dataset, batch_size, sampler=sampler)
    return dataloader

In [8]:
seq_length = 60
train_dataset = CustomDataset(X_train, sequence_len=seq_length, y=y_train)
valid_dataset = CustomDataset(X_test, sequence_len=seq_length, y=y_test)
test_dataset = CustomDataset(test, sequence_len=seq_length, mode="test")

train_dataloader = CustomDataloader(
    train_dataset, X_train.shape[0], seq_length, X_train.shape[1], 512
)
valid_dataloader = CustomDataloader(
    valid_dataset, X_test.shape[0], seq_length, X_test.shape[1], 512
)
test_dataloader = CustomDataloader(
    test_dataset, test.shape[0], seq_length, test.shape[1], 512
)


In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [10]:
class LSTM(nn.Module):
    def __init__(self,input_size,num_classes,hidden_size,num_layers):
        super().__init__()
        
        self.lstm1 = nn.LSTM(input_size,hidden_size,num_layers,batch_first=True,dropout=0,bidirectional=True)

        self.final = nn.Sequential(
            nn.ReLU(),
            nn.Linear(hidden_size*60*2, num_classes),
        )
            
    def forward(self,x):
        
        out, _ = self.lstm1(x)
        
        out = out.reshape(out.shape[0],-1)
        
        out = self.final(out)
        return out

In [11]:
input_size = len(features)
hidden_size = 128
num_classes = 1
learning_rate = 1e-4
num_epochs = 100

model = LSTM(input_size,num_classes,hidden_size,1)
model.to(device)

LSTM(
  (lstm1): LSTM(13, 128, batch_first=True, bidirectional=True)
  (final): Sequential(
    (0): ReLU()
    (1): Linear(in_features=15360, out_features=1, bias=True)
  )
)

In [12]:

criterion = nn.BCEWithLogitsLoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
num_warmup_steps = int(0.1 * num_epochs * len(train_dataloader))
num_training_steps = int(num_epochs * len(train_dataloader))
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps, num_training_steps
)

In [13]:
torch.cuda.empty_cache()
for epoch in tqdm(range(num_epochs-1)): 
    model.train()
    for trainX, train_y in train_dataloader:
        outputs = model(trainX.to(device,dtype=torch.float32)).squeeze(-1)
        optimizer.zero_grad()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        loss = criterion(outputs, train_y.to(device,dtype=torch.float32))
        loss.backward()

        optimizer.step()
        scheduler.step()

    model.eval()
    for validX, valid_y in valid_dataloader:
        with torch.no_grad():
            val_out = model(validX.to(device,dtype=torch.float32)).squeeze(-1)
            vall_loss = criterion(val_out,valid_y.to(device,dtype=torch.float32))

    if epoch % 10 == 0:
        
          print("Epoch: %d, loss: %1.5f valid loss:  %1.5f " %(epoch, loss.cpu().item(),vall_loss.cpu().item()))

  1%|          | 1/99 [00:01<03:00,  1.85s/it]

Epoch: 0, loss: 0.69331 valid loss:  0.69200 


  8%|▊         | 8/99 [00:11<02:08,  1.41s/it]